# Import Packages and Codes

In [ ]:
%load_ext google.cloud.bigquery
from datetime import date
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil
from IPython.display import display, HTML
import os
import subprocess

In [ ]:
dataset = os.getenv("WORKSPACE_CDR")
dataset

In [ ]:
cwd = os.getcwd()
cwd

# Find Condition Codes

In [ ]:
#Acidosis
acidosis = {'9': ['276.2'], 
       '10': ['E87.2']}

# Build Cohort

## Obtain Demographics Data from Google Cloud Storage

In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'demographic_all.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
demo_patients = pd.read_csv(name_of_file_in_bucket)
demo_patients

## Obtain Patient List of Acidosis

In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'metformin_acidosis_AG_10132023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
my_dataframe = pd.read_csv(name_of_file_in_bucket)
my_dataframe.head()

In [ ]:
person_list = my_dataframe['person_id'].tolist()
person_list

## Function to Query Visit Occurrence by Person_id from Acidosis Extraction

In [ ]:
#Query person ids and condition start dates by specific ICD 9&10 diagnosis codes.
def query_by_person():  
    query = ("""
                SELECT *
                FROM 
                    `"""+dataset+""".ds_visit_occurrence` AS vo
                WHERE
                    vo.person_id IN ("""+",".join([str(pid) for pid in person_list])+""")
    """)
    df_condition= pd.read_gbq(query, dialect="standard")
    return df_condition

In [ ]:
df_visit = query_by_person()
df_visit

## Upload Dataset to Google Bucket

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = df_visit

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'Acidosis_Visit_Occurrence_AG_10162023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr

In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'Acidosis_Visit_Occurrence_AG_10162023.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
my_dataframe = pd.read_csv(name_of_file_in_bucket)
my_dataframe.head()

In [ ]:
test = my_dataframe[['STANDARD_CONCEPT_NAME','STANDARD_CONCEPT_CODE']].drop_duplicates()

In [ ]:
test.reset_index()

In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))

# Visualizations

In [ ]:
cleaned_demo = df_visit.rename(columns={'person_id':"Count",
                                            'year_of_birth':"Age",
                                            'gender_source_value': "Gender",
                                            'sex_at_birth_source_value':"Sex at Birth",
                                            'race_source_value':"Race",
                                            'ethnicity_source_value':"Hispanic"})

In [ ]:
cleaned_demo.head(5)

In [ ]:
count_visit_gr = cleaned_demo[['PERSON_ID','STANDARD_CONCEPT_NAME']].groupby(['STANDARD_CONCEPT_NAME'], as_index=False).count()
count_visit_gr['%'] = 100 * count_visit_gr['PERSON_ID'] / len(cleaned_demo)
display(count_visit_gr)

sns.barplot(x='PERSON_ID', y='STANDARD_CONCEPT_NAME',data=cleaned_demo[['PERSON_ID','STANDARD_CONCEPT_NAME']].groupby(['STANDARD_CONCEPT_NAME'], as_index=False).count());